<a href="https://colab.research.google.com/github/chalika-vanyaa/Halodoc_Sentiment_Analysis_Project/blob/main/Chalika_Vanya_Resya_Sentiment_Analysis_Project_Model_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Import Library**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import nltk
nltk.download('punkt_tab')
nltk.download('stopwords')

!pip install sastrawi

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 4.2 MB/s eta 0:00:00


In [ ]:
# requirements.txt
!pip freeze requirements.txt

absl-py==1.4.0
accelerate==1.2.1
aiohappyeyeballs==2.4.4
aiohttp==3.11.11
aiosignal==1.3.2
alabaster==1.0.0
albucore==0.0.19
albumentations==1.4.20
altair==5.5.0
annotated-types==0.7.0
anyio==3.7.1
argon2-cffi==23.1.0
argon2-cffi-bindings==21.2.0
array_record==0.6.0
arviz==0.20.0
astropy==6.1.7
astropy-iers-data==0.2025.1.6.0.33.42
astunparse==1.6.3
atpublic==4.1.0
attrs==24.3.0
audioread==3.0.1
autograd==1.7.0
babel==2.16.0
backcall==0.2.0
beautifulsoup4==4.12.3
bigframes==1.31.0
bigquery-magics==0.5.0
bleach==6.2.0
blinker==1.9.0
blis==0.7.11
blosc2==3.0.0
bokeh==3.6.2
Bottleneck==1.4.2
bqplot==0.12.44
branca==0.8.1
CacheControl==0.14.2
cachetools==5.5.0
catalogue==2.0.10
certifi==2024.12.14
cffi==1.17.1
chardet==5.2.0
charset-normalizer==3.4.1
chex==0.1.88
clarabel==0.9.0
click==8.1.8
cloudpathlib==0.20.0
cloudpickle==3.1.0
cmake==3.31.2
cmdstanpy==1.2.5
colorcet==3.1.0
colorlover==0.3.0
colour==0.1.5
community==1.0.0b1
confection==0.1.5
cons==0.4.6
contourpy==1.3.1
cryptography==43

## **Scraping Dataset**

In [ ]:
!pip install google-play-scraper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 1.9 MB/s eta 0:00:00


In [ ]:
# scraping halodoc review data from google play store
from google_play_scraper import reviews, app, Sort
halodoc_app_review, continuation_token = reviews(
    'com.linkdokter.halodoc.android',
    lang='id',
    country='id',
    sort=Sort.MOST_RELEVANT,
    count=11000 #11.000 data
)

halodoc_app_review, _ = reviews(
    'com.linkdokter.halodoc.android',
    continuation_token=continuation_token
)

In [ ]:
# checking data length
len(halodoc_app_review)

11000

In [ ]:
# showing the first 3 reviews
halodoc_app_review[:3]

[{'reviewId': '8c7deccc-c422-478a-9f73-5ee938b1057f',
  'userName': 'Pengguna Google',
  'userImage': 'https://play-lh.googleusercontent.com/EGemoI2NTXmTsBVtJqk8jxF9rh8ApRWfsIMQSt2uE4OcpQqbFu7f7NbTK05lx80nuSijCz7sc3a277R67g',
  'content': 'Baru nyobain pertama kali pesen obat lewat halodoc. Waw secepat dan semudah itu proses pemesanan nya. 👍 Petunjuk cara pemesanan di aplikasinya lengkap dan pokoknya sip deh Makasih yaa Halodoc',
  'score': 5,
  'thumbsUpCount': 0,
  'reviewCreatedVersion': '6.600',
  'at': datetime.datetime(2020, 2, 15, 5, 7, 7),
  'replyContent': 'Hai nadiyah fatya! Terima kasih sudah menggunakan Halodoc. Jangan lupa untuk jaga kesehatanmu dengan Halodoc. Semoga kamu sehat selalu!',
  'repliedAt': datetime.datetime(2020, 2, 15, 5, 23, 9),
  'appVersion': '6.600'},
 {'reviewId': '579eafaf-2cd2-4238-9fe1-f144b7491635',
  'userName': 'Pengguna Google',
  'userImage': 'https://play-lh.googleusercontent.com/EGemoI2NTXmTsBVtJqk8jxF9rh8ApRWfsIMQSt2uE4OcpQqbFu7f7NbTK05lx80nu

In [ ]:
# creating a dataframe based on the scraped data
review_data = pd.DataFrame(halodoc_app_review).sort_values(by='at', ascending=False).reset_index(drop=True)
review_data

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion
0,542712a1-53d6-4a47-8318-ad7c7d806f10,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Dokternya baik,5,0,23.910,2025-01-21 01:11:20,Hai Manjadda Wajada! Terima kasih sudah menggu...,2025-01-21 07:57:32,23.910
1,2b2301ab-6b4f-4a12-b384-db28ff9ff162,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Dokter yg ramah ramah dan obat yg di anjurkan ...,5,0,None,2025-01-21 00:38:40,Hai denis saputra! Terima kasih sudah mengguna...,2025-01-21 04:32:56,None
2,9552215a-324b-4e7c-81ac-e2ef212a93bf,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Deskripsi dokter kurang detail .. masak sudah ...,2,0,None,2025-01-18 11:50:03,Hai idrus! Mohon maaf atas ketidaknyamanannya....,2025-01-18 13:28:54,None
3,89eebf24-773e-424a-bb75-a0a486d4f313,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Konsul dengan dokter M.Ridwan hakiki enak bgt ...,5,0,None,2025-01-16 02:22:16,Hai Misran BIN Terima kasih sudah menggunakan ...,2025-01-16 03:55:28,None
4,00ab49f9-f844-423f-8892-f43939e23d65,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Responsif dan bagus pelayanannya,5,0,None,2025-01-15 22:11:22,Hai Heriyanto! Terima kasih sudah menggunakan ...,2025-01-15 23:27:36,None
...,...,...,...,...,...,...,...,...,...,...,...
10995,0bae96f4-2eb0-4451-adcf-1ba3d5069585,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Aplikasi yg bagus karena langsung di jwb sama ...,5,0,3.220,2018-09-13 09:04:01,Hai Natalia Permata! Terima kasih sudah menggu...,2018-09-13 09:40:47,3.220
10996,db8f7e55-86e0-4938-8e19-9bd849a3df9e,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Terimaksih sudah membuat aplikasi yg berguna d...,5,0,3.000,2018-09-13 07:08:03,Hai! Terima kasih sudah menggunakan Halodoc. J...,2018-09-13 07:22:40,3.000
10997,b56fe98c-62ff-462a-948d-3314b8a06316,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Sangat membantu mendekatkan masyarakat dengan ...,5,0,3.220,2018-09-13 02:45:53,Hai Eko Yudhianto! Terima kasih sudah mengguna...,2018-09-13 04:14:28,3.220
10998,f472a08e-3c5d-4f91-bc93-35df7856a147,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"Terimakasih, Sangat membantu, semoga kedepanny...",5,0,3.220,2018-09-12 20:42:15,Hai Ardiaz Atmadeva! Terima kasih sudah menggu...,2018-09-12 21:08:07,3.220


In [ ]:
# saving some features only to make the data more concise
review_data = review_data[['userName', 'content', 'score', 'at']]
review_data

,userName,content,score,at
0,Pengguna Google,Dokternya baik,5,2025-01-21 01:11:20
1,Pengguna Google,Dokter yg ramah ramah dan obat yg di anjurkan ...,5,2025-01-21 00:38:40
2,Pengguna Google,Deskripsi dokter kurang detail .. masak sudah ...,2,2025-01-18 11:50:03
3,Pengguna Google,Konsul dengan dokter M.Ridwan hakiki enak bgt ...,5,2025-01-16 02:22:16
4,Pengguna Google,Responsif dan bagus pelayanannya,5,2025-01-15 22:11:22
...,...,...,...,...
10995,Pengguna Google,Aplikasi yg bagus karena langsung di jwb sama ...,5,2018-09-13 09:04:01
10996,Pengguna Google,Terimaksih sudah membuat aplikasi yg berguna d...,5,2018-09-13 07:08:03
10997,Pengguna Google,Sangat membantu mendekatkan masyarakat dengan ...,5,2018-09-13 02:45:53
10998,Pengguna Google,"Terimakasih, Sangat membantu, semoga kedepanny...",5,2018-09-12 20:42:15


In [ ]:
# save data into csv file
review_data.to_csv('halodoc_playstore_review.csv', index=False)

## **Data Understanding**

In [ ]:
# data loading
review_data = pd.read_csv('halodoc_playstore_review.csv')
review_data.head()

,userName,content,score,at
0,Pengguna Google,Dokternya baik,5,2025-01-21 01:11:20
1,Pengguna Google,Dokter yg ramah ramah dan obat yg di anjurkan ...,5,2025-01-21 00:38:40
2,Pengguna Google,Deskripsi dokter kurang detail .. masak sudah ...,2,2025-01-18 11:50:03
3,Pengguna Google,Konsul dengan dokter M.Ridwan hakiki enak bgt ...,5,2025-01-16 02:22:16
4,Pengguna Google,Responsif dan bagus pelayanannya,5,2025-01-15 22:11:22


In [ ]:
# basic information of data
review_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11000 entries, 0 to 10999
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   userName  11000 non-null  object
 1   content   11000 non-null  object
 2   score     11000 non-null  int64 
 3   at        11000 non-null  object
dtypes: int64(1), object(3)
memory usage: 343.9+ KB


In [ ]:
# checking missing values
review_data.isnull().sum()

,0
userName,0
content,0
score,0
at,0


In [ ]:
# checking duplicated values
review_data.duplicated().sum()

0

## **Text Preprocessing**

In [ ]:
import re
import string

# function to clean text by removing numbers, special characters, and any redundancies
def text_cleaning(review):
  cleaned_text = re.sub(r'[^A-Za-z\s]', '', review) # removing numbers and other special chars including punctuation
  cleaned_text = cleaned_text.strip() # removing whitespaces
  cleaned_text = cleaned_text.replace('\n', ' ') # replace new line with spaces
  return cleaned_text

In [ ]:
# function to transform text into lowercase
def text_lowercase(review):
  lowercase_text = review.lower()
  return lowercase_text

In [ ]:
from nltk.tokenize import word_tokenize

# function to tokenize words
def tokenization(review):
  tokens = word_tokenize(review)
  return tokens

In [ ]:
# list of indonesian's common slangwords
slangwords = {"@": "di", "abis": "habis", "wtb": "beli", "masi": "masih", "wts": "jual", "wtt": "tukar", "bgt": "banget", "maks": "maksimal", "plisss": "tolong", "bgttt": "banget", "bgt": "banget", "pingin": "ingin", "indo": "indonesia", "bgtt": "banget", "ad": "ada", "rv": "redvelvet", "plis": "tolong", "pls": "tolong", "cr": "sumber", "cod": "bayar ditempat", "adlh": "adalah", "afaik": "as far as i know", "ahaha": "haha", "aj": "saja", "ajep-ajep": "dunia gemerlap", "ak": "saya", "akika": "aku", "akkoh": "aku", "akuwh": "aku", "alay": "norak", "alow": "halo", "ambilin": "ambilkan", "ancur": "hancur", "anjrit": "anjing", "anter": "antar", "ap2": "apa-apa", "apasih": "apa sih", "apes": "sial", "aps": "apa", "aq": "saya", "aquwh": "aku", "asbun": "asal bunyi", "aseekk": "asyik", "asekk": "asyik", "asem": "asam", "aspal": "asli tetapi palsu", "astul": "asal tulis", "ato": "atau", "au ah": "tidak mau tahu", "awak": "saya", "ay": "sayang", "ayank": "sayang", "b4": "sebelum", "bakalan": "akan", "bandes": "bantuan desa", "bangedh": "banget", "banpol": "bantuan polisi", "banpur": "bantuan tempur", "basbang": "basi", "bcanda": "bercanda", "bdg": "bandung", "begajulan": "nakal", "beliin": "belikan", "bencong": "banci", "bentar": "sebentar", "ber3": "bertiga", "beresin": "membereskan", "bete": "bosan", "beud": "banget", "bg": "abang", "bgmn": "bagaimana", "bgt": "banget", "bijimane": "bagaimana", "bintal": "bimbingan mental", "bkl": "akan", "bknnya": "bukannya", "blegug": "bodoh", "blh": "boleh", "bln": "bulan", "blum": "belum", "bnci": "benci", "bnran": "yang benar", "bodor": "lucu", "bokap": "ayah", "boker": "buang air besar", "bokis": "bohong", "boljug": "boleh juga", "bonek": "bocah nekat", "boyeh": "boleh", "br": "baru", "brg": "bareng", "bro": "saudara laki-laki", "bru": "baru", "bs": "bisa", "bsen": "bosan", "bt": "buat", "btw": "ngomong-ngomong", "buaya": "tidak setia", "bubbu": "tidur", "bubu": "tidur", "bumil": "ibu hamil", "bw": "bawa", "bwt": "buat", "byk": "banyak", "byrin": "bayarkan", "cabal": "sabar", "cadas": "keren", "calo": "makelar", "can": "belum", "capcus": "pergi", "caper": "cari perhatian", "ce": "cewek", "cekal": "cegah tangkal", "cemen": "penakut", "cengengesan": "tertawa", "cepet": "cepat", "cew": "cewek", "chuyunk": "sayang", "cimeng": "ganja", "cipika cipiki": "cium pipi kanan cium pipi kiri", "ciyh": "sih", "ckepp": "cakep", "ckp": "cakep", "cmiiw": "correct me if i'm wrong", "cmpur": "campur", "cong": "banci", "conlok": "cinta lokasi", "cowwyy": "maaf", "cp": "siapa", "cpe": "capek", "cppe": "capek", "cucok": "cocok", "cuex": "cuek", "cumi": "Cuma miscall", "cups": "culun", "curanmor": "pencurian kendaraan bermotor", "curcol": "curahan hati colongan", "cwek": "cewek", "cyin": "cinta", "d": "di", "dah": "deh", "dapet": "dapat", "de": "adik", "dek": "adik", "demen": "suka", "deyh": "deh", "dgn": "dengan", "diancurin": "dihancurkan", "dimaafin": "dimaafkan", "dimintak": "diminta", "disono": "di sana", "dket": "dekat", "dkk": "dan kawan-kawan", "dll": "dan lain-lain", "dlu": "dulu", "dngn": "dengan", "dodol": "bodoh", "doku": "uang", "dongs": "dong", "dpt": "dapat", "dri": "dari", "drmn": "darimana", "drtd": "dari tadi", "dst": "dan seterusnya", "dtg": "datang", "duh": "aduh", "duren": "durian", "ed": "edisi", "egp": "emang gue pikirin", "eke": "aku", "elu": "kamu", "emangnya": "memangnya", "emng": "memang", "endak": "tidak", "enggak": "tidak", "envy": "iri", "ex": "mantan", "fax": "facsimile", "fifo": "first in first out", "folbek": "follow back", "fyi": "sebagai informasi", "gaada": "tidak ada uang", "gag": "tidak", "gaje": "tidak jelas", "gak papa": "tidak apa-apa", "gan": "juragan", "gaptek": "gagap teknologi", "gatek": "gagap teknologi", "gawe": "kerja", "gbs": "tidak bisa", "gebetan": "orang yang disuka", "geje": "tidak jelas", "gepeng": "gelandangan dan pengemis", "ghiy": "lagi", "gile": "gila", "gimana": "bagaimana", "gino": "gigi nongol", "githu": "gitu", "gj": "tidak jelas", "gmana": "bagaimana", "gn": "begini", "goblok": "bodoh", "golput": "golongan putih", "gowes": "mengayuh sepeda", "gpny": "tidak punya", "gr": "gede rasa", "gretongan": "gratisan", "gtau": "tidak tahu", "gua": "saya", "guoblok": "goblok", "gw": "saya", "ha": "tertawa", "haha": "tertawa", "hallow": "halo", "hankam": "pertahanan dan keamanan", "hehe": "he", "helo": "halo", "hey": "hai", "hlm": "halaman", "hny": "hanya", "hoax": "isu bohong", "hr": "hari", "hrus": "harus", "hubdar": "perhubungan darat", "huff": "mengeluh", "hum": "rumah", "humz": "rumah", "ilang": "hilang", "ilfil": "tidak suka", "imho": "in my humble opinion", "imoetz": "imut", "item": "hitam", "itungan": "hitungan", "iye": "iya", "ja": "saja", "jadiin": "jadi", "jaim": "jaga image", "jayus": "tidak lucu", "jdi": "jadi", "jem": "jam", "jga": "juga", "jgnkan": "jangankan", "jir": "anjing", "jln": "jalan", "jomblo": "tidak punya pacar", "jubir": "juru bicara", "jutek": "galak", "k": "ke", "kab": "kabupaten", "kabor": "kabur", "kacrut": "kacau", "kadiv": "kepala divisi", "kagak": "tidak", "kalo": "kalau", "kampret": "sialan", "kamtibmas": "keamanan dan ketertiban masyarakat", "kamuwh": "kamu", "kanwil": "kantor wilayah", "karna": "karena", "kasubbag": "kepala subbagian", "katrok": "kampungan", "kayanya": "kayaknya", "kbr": "kabar", "kdu": "harus", "kec": "kecamatan", "kejurnas": "kejuaraan nasional", "kekeuh": "keras kepala", "kel": "kelurahan", "kemaren": "kemarin", "kepengen": "mau", "kepingin": "mau", "kepsek": "kepala sekolah", "kesbang": "kesatuan bangsa", "kesra": "kesejahteraan rakyat", "ketrima": "diterima", "kgiatan": "kegiatan", "kibul": "bohong", "kimpoi": "kawin", "kl": "kalau", "klianz": "kalian", "kloter": "kelompok terbang", "klw": "kalau", "km": "kamu", "kmps": "kampus", "kmrn": "kemarin", "knal": "kenal", "knp": "kenapa", "kodya": "kota madya", "komdis": "komisi disiplin", "komsov": "komunis sovyet", "kongkow": "kumpul bareng teman-teman", "kopdar": "kopi darat", "korup": "korupsi", "kpn": "kapan", "krenz": "keren", "krm": "kirim", "kt": "kita", "ktmu": "ketemu", "ktr": "kantor", "kuper": "kurang pergaulan", "kw": "imitasi", "kyk": "seperti", "la": "lah", "lam": "salam", "lamp": "lampiran", "lanud": "landasan udara", "latgab": "latihan gabungan", "lebay": "berlebihan", "leh": "boleh", "lelet": "lambat", "lemot": "lambat", "lgi": "lagi", "lgsg": "langsung", "liat": "lihat", "litbang": "penelitian dan pengembangan", "lmyn": "lumayan", "lo": "kamu", "loe": "kamu", "lola": "lambat berfikir", "louph": "cinta", "low": "kalau", "lp": "lupa", "luber": "langsung, umum, bebas, dan rahasia", "luchuw": "lucu", "lum": "belum", "luthu": "lucu", "lwn": "lawan", "maacih": "terima kasih", "mabal": "bolos", "macem": "macam", "macih": "masih", "maem": "makan", "magabut": "makan gaji buta", "maho": "homo", "mak jang": "kaget", "maksain": "memaksa", "malem": "malam", "mam": "makan", "maneh": "kamu", "maniez": "manis", "mao": "mau", "masukin": "masukkan", "melu": "ikut", "mepet": "dekat sekali", "mgu": "minggu", "migas": "minyak dan gas bumi", "mikol": "minuman beralkohol", "miras": "minuman keras", "mlah": "malah", "mngkn": "mungkin", "mo": "mau", "mokad": "mati", "moso": "masa", "mpe": "sampai", "msk": "masuk", "mslh": "masalah", "mt": "makan teman", "mubes": "musyawarah besar", "mulu": "melulu", "mumpung": "selagi", "munas": "musyawarah nasional", "muntaber": "muntah dan berak", "musti": "mesti", "muupz": "maaf", "mw": "now watching", "n": "dan", "nanam": "menanam", "nanya": "bertanya", "napa": "kenapa", "napi": "narapidana", "napza": "narkotika, alkohol, psikotropika, dan zat adiktif ", "narkoba": "narkotika, psikotropika, dan obat terlarang", "nasgor": "nasi goreng", "nda": "tidak", "ndiri": "sendiri", "ne": "ini", "nekolin": "neokolonialisme", "nembak": "menyatakan cinta", "ngabuburit": "menunggu berbuka puasa", "ngaku": "mengaku", "ngambil": "mengambil", "nganggur": "tidak punya pekerjaan", "ngapah": "kenapa", "ngaret": "terlambat", "ngasih": "memberikan", "ngebandel": "berbuat bandel", "ngegosip": "bergosip", "ngeklaim": "mengklaim", "ngeksis": "menjadi eksis", "ngeles": "berkilah", "ngelidur": "menggigau", "ngerampok": "merampok", "ngga": "tidak", "ngibul": "berbohong", "ngiler": "mau", "ngiri": "iri", "ngisiin": "mengisikan", "ngmng": "bicara", "ngomong": "bicara", "ngubek2": "mencari-cari", "ngurus": "mengurus", "nie": "ini", "nih": "ini", "niyh": "nih", "nmr": "nomor", "nntn": "nonton", "nobar": "nonton bareng", "np": "now playing", "ntar": "nanti", "ntn": "nonton", "numpuk": "bertumpuk", "nutupin": "menutupi", "nyari": "mencari", "nyekar": "menyekar", "nyicil": "mencicil", "nyoblos": "mencoblos", "nyokap": "ibu", "ogah": "tidak mau", "ol": "online", "ongkir": "ongkos kirim", "oot": "out of topic", "org2": "orang-orang", "ortu": "orang tua", "otda": "otonomi daerah", "otw": "on the way, sedang di jalan", "pacal": "pacar", "pake": "pakai", "pala": "kepala", "pansus": "panitia khusus", "parpol": "partai politik", "pasutri": "pasangan suami istri", "pd": "pada", "pede": "percaya diri", "pelatnas": "pemusatan latihan nasional", "pemda": "pemerintah daerah", "pemkot": "pemerintah kota", "pemred": "pemimpin redaksi", "penjas": "pendidikan jasmani", "perda": "peraturan daerah", "perhatiin": "perhatikan", "pesenan": "pesanan", "pgang": "pegang", "pi": "tapi", "pilkada": "pemilihan kepala daerah", "pisan": "sangat", "pk": "penjahat kelamin", "plg": "paling", "pmrnth": "pemerintah", "polantas": "polisi lalu lintas", "ponpes": "pondok pesantren", "pp": "pulang pergi", "prg": "pergi", "prnh": "pernah", "psen": "pesan", "pst": "pasti", "pswt": "pesawat", "pw": "posisi nyaman", "qmu": "kamu", "rakor": "rapat koordinasi", "ranmor": "kendaraan bermotor", "re": "reply", "ref": "referensi", "rehab": "rehabilitasi", "rempong": "sulit", "repp": "balas", "restik": "reserse narkotika", "rhs": "rahasia", "rmh": "rumah", "ru": "baru", "ruko": "rumah toko", "rusunawa": "rumah susun sewa", "ruz": "terus", "saia": "saya", "salting": "salah tingkah", "sampe": "sampai", "samsek": "sama sekali", "sapose": "siapa", "satpam": "satuan pengamanan", "sbb": "sebagai berikut", "sbh": "sebuah", "sbnrny": "sebenarnya", "scr": "secara", "sdgkn": "sedangkan", "sdkt": "sedikit", "se7": "setuju", "sebelas dua belas": "mirip", "sembako": "sembilan bahan pokok", "sempet": "sempat", "sendratari": "seni drama tari", "sgt": "sangat", "shg": "sehingga", "siech": "sih", "sikon": "situasi dan kondisi", "sinetron": "sinema elektronik", "siramin": "siramkan", "sj": "saja", "skalian": "sekalian", "sklh": "sekolah", "skt": "sakit", "slesai": "selesai", "sll": "selalu", "slma": "selama", "slsai": "selesai", "smpt": "sempat", "smw": "semua", "sndiri": "sendiri", "soljum": "sholat jumat", "songong": "sombong", "sory": "maaf", "sosek": "sosial-ekonomi", "sotoy": "sok tahu", "spa": "siapa", "sppa": "siapa", "spt": "seperti", "srtfkt": "sertifikat", "stiap": "setiap", "stlh": "setelah", "suk": "masuk", "sumpek": "sempit", "syg": "sayang", "t4": "tempat", "tajir": "kaya", "tau": "tahu", "taw": "tahu", "td": "tadi", "tdk": "tidak", "teh": "kakak perempuan", "telat": "terlambat", "telmi": "telat berpikir", "temen": "teman", "tengil": "menyebalkan", "tepar": "terkapar", "tggu": "tunggu", "tgu": "tunggu", "thankz": "terima kasih", "thn": "tahun", "tilang": "bukti pelanggaran", "tipiwan": "TvOne", "tks": "terima kasih", "tlp": "telepon", "tls": "tulis", "tmbah": "tambah", "tmen2": "teman-teman", "tmpah": "tumpah", "tmpt": "tempat", "tngu": "tunggu", "tnyta": "ternyata", "tokai": "tai", "toserba": "toko serba ada", "tpi": "tapi", "trdhulu": "terdahulu", "trima": "terima kasih", "trm": "terima", "trs": "terus", "trutama": "terutama", "ts": "penulis", "tst": "tahu sama tahu", "ttg": "tentang", "tuch": "tuh", "tuir": "tua", "tw": "tahu", "u": "kamu", "ud": "sudah", "udah": "sudah", "ujg": "ujung", "ul": "ulangan", "unyu": "lucu", "uplot": "unggah", "urang": "saya", "usah": "perlu", "utk": "untuk", "valas": "valuta asing", "w/": "dengan", "wadir": "wakil direktur", "wamil": "wajib militer", "warkop": "warung kopi", "warteg": "warung tegal", "wat": "buat", "wkt": "waktu", "wtf": "what the fuck", "xixixi": "tertawa", "ya": "iya", "yap": "iya", "yaudah": "ya sudah", "yawdah": "ya sudah", "yg": "yang", "yl": "yang lain", "yo": "iya", "yowes": "ya sudah", "yup": "iya", "7an": "tujuan", "ababil": "abg labil", "acc": "accord", "adlah": "adalah", "adoh": "aduh", "aha": "tertawa", "aing": "saya", "aja": "saja", "ajj": "saja", "aka": "dikenal juga sebagai", "akko": "aku", "akku": "aku", "akyu": "aku", "aljasa": "asal jadi saja", "ama": "sama", "ambl": "ambil", "anjir": "anjing", "ank": "anak", "ap": "apa", "apaan": "apa", "ape": "apa", "aplot": "unggah", "apva": "apa", "aqu": "aku", "asap": "sesegera mungkin", "aseek": "asyik", "asek": "asyik", "aseknya": "asyiknya", "asoy": "asyik", "astrojim": "astagfirullahaladzim", "ath": "kalau begitu", "atuh": "kalau begitu", "ava": "avatar", "aws": "awas", "ayang": "sayang", "ayok": "ayo", "bacot": "banyak bicara", "bales": "balas", "bangdes": "pembangunan desa", "bangkotan": "tua", "banpres": "bantuan presiden", "bansarkas": "bantuan sarana kesehatan", "bazis": "badan amal, zakat, infak, dan sedekah", "bcoz": "karena", "beb": "sayang", "bejibun": "banyak", "belom": "belum", "bener": "benar", "ber2": "berdua", "berdikari": "berdiri di atas kaki sendiri", "bet": "banget", "beti": "beda tipis", "beut": "banget", "bgd": "banget", "bgs": "bagus", "bhubu": "tidur", "bimbuluh": "bimbingan dan penyuluhan", "bisi": "kalau-kalau", "bkn": "bukan", "bl": "beli", "blg": "bilang", "blm": "belum", "bls": "balas", "bnchi": "benci", "bngung": "bingung", "bnyk": "banyak", "bohay": "badan aduhai", "bokep": "porno", "bokin": "pacar", "bole": "boleh", "bolot": "bodoh", "bonyok": "ayah ibu", "bpk": "bapak", "brb": "segera kembali", "brngkt": "berangkat", "brp": "berapa", "brur": "saudara laki-laki", "bsa": "bisa", "bsk": "besok", "bu_bu": "tidur", "bubarin": "bubarkan", "buber": "buka bersama", "bujubune": "luar biasa", "buser": "buru sergap", "bwhn": "bawahan", "byar": "bayar", "byr": "bayar", "c8": "chat", "cabut": "pergi", "caem": "cakep", "cama-cama": "sama-sama", "cangcut": "celana dalam", "cape": "capek", "caur": "jelek", "cekak": "tidak ada uang", "cekidot": "coba lihat", "cemplungin": "cemplungkan", "ceper": "pendek", "ceu": "kakak perempuan", "cewe": "cewek", "cibuk": "sibuk", "cin": "cinta", "ciye": "cie", "ckck": "ck", "clbk": "cinta lama bersemi kembali", "cmpr": "campur", "cnenk": "senang", "congor": "mulut", "cow": "cowok", "coz": "karena", "cpa": "siapa", "gokil": "gila", "gombal": "suka merayu", "gpl": "tidak pakai lama", "gpp": "tidak apa-apa", "gretong": "gratis", "gt": "begitu", "gtw": "tidak tahu", "gue": "saya", "guys": "teman-teman", "gws": "cepat sembuh", "haghaghag": "tertawa", "hakhak": "tertawa", "handak": "bahan peledak", "hansip": "pertahanan sipil", "hellow": "halo", "helow": "halo", "hi": "hai", "hlng": "hilang", "hnya": "hanya", "houm": "rumah", "hrs": "harus", "hubad": "hubungan angkatan darat", "hubla": "perhubungan laut", "huft": "mengeluh", "humas": "hubungan masyarakat", "idk": "saya tidak tahu", "ilfeel": "tidak suka", "imba": "jago sekali", "imoet": "imut", "info": "informasi", "itung": "hitung", "isengin": "bercanda", "iyala": "iya lah", "iyo": "iya", "jablay": "jarang dibelai", "jadul": "jaman dulu", "jancuk": "anjing", "jd": "jadi", "jdikan": "jadikan", "jg": "juga", "jgn": "jangan", "jijay": "jijik", "jkt": "jakarta", "jnj": "janji", "jth": "jatuh", "jurdil": "jujur adil", "jwb": "jawab", "ka": "kakak", "kabag": "kepala bagian", "kacian": "kasihan", "kadit": "kepala direktorat", "kaga": "tidak", "kaka": "kakak", "kamtib": "keamanan dan ketertiban", "kamuh": "kamu", "kamyu": "kamu", "kapt": "kapten", "kasat": "kepala satuan", "kasubbid": "kepala subbidang", "kau": "kamu", "kbar": "kabar", "kcian": "kasihan", "keburu": "terlanjur", "kedubes": "kedutaan besar", "kek": "seperti", "keknya": "kayaknya", "keliatan": "kelihatan", "keneh": "masih", "kepikiran": "terpikirkan", "kepo": "mau tahu urusan orang", "kere": "tidak punya uang", "kesian": "kasihan", "ketauan": "ketahuan", "keukeuh": "keras kepala", "khan": "kan", "kibus": "kaki busuk", "kk": "kakak", "klian": "kalian", "klo": "kalau", "kluarga": "keluarga", "klwrga": "keluarga", "kmari": "kemari", "kmpus": "kampus", "kn": "kan", "knl": "kenal", "knpa": "kenapa", "kog": "kok", "kompi": "komputer", "komtiong": "komunis Tiongkok", "konjen": "konsulat jenderal", "koq": "kok", "kpd": "kepada", "kptsan": "keputusan", "krik": "garing", "krn": "karena", "ktauan": "ketahuan", "ktny": "katanya", "kudu": "harus", "kuq": "kok", "ky": "seperti", "kykny": "kayanya", "laka": "kecelakaan", "lambreta": "lambat", "lansia": "lanjut usia", "lapas": "lembaga pemasyarakatan", "lbur": "libur", "lekong": "laki-laki", "lg": "lagi", "lgkp": "lengkap", "lht": "lihat", "linmas": "perlindungan masyarakat", "lmyan": "lumayan", "lngkp": "lengkap", "loch": "loh", "lol": "tertawa", "lom": "belum", "loupz": "cinta", "lowh": "kamu", "lu": "kamu", "luchu": "lucu", "luff": "cinta", "luph": "cinta", "lw": "kamu", "lwt": "lewat", "maaciw": "terima kasih", "mabes": "markas besar", "macem-macem": "macam-macam", "madesu": "masa depan suram", "maen": "main", "mahatma": "maju sehat bersama", "mak": "ibu", "makasih": "terima kasih", "malah": "bahkan", "malu2in": "memalukan", "mamz": "makan", "manies": "manis", "mantep": "mantap", "markus": "makelar kasus", "mba": "mbak", "mending": "lebih baik", "mgkn": "mungkin", "mhn": "mohon", "miker": "minuman keras", "milis": "mailing list", "mksd": "maksud", "mls": "malas", "mnt": "minta", "moge": "motor gede", "mokat": "mati", "mosok": "masa", "msh": "masih", "mskpn": "meskipun", "msng2": "masing-masing", "muahal": "mahal", "muker": "musyawarah kerja", "mumet": "pusing", "muna": "munafik", "munaslub": "musyawarah nasional luar biasa", "musda": "musyawarah daerah", "muup": "maaf", "muuv": "maaf", "nal": "kenal", "nangis": "menangis", "naon": "apa", "napol": "narapidana politik", "naq": "anak", "narsis": "bangga pada diri sendiri", "nax": "anak", "ndak": "tidak", "ndut": "gendut", "nekolim": "neokolonialisme", "nelfon": "menelepon", "ngabis2in": "menghabiskan", "ngakak": "tertawa", "ngambek": "marah", "ngampus": "pergi ke kampus", "ngantri": "mengantri", "ngapain": "sedang apa", "ngaruh": "berpengaruh", "ngawur": "berbicara sembarangan", "ngeceng": "kumpul bareng-bareng", "ngeh": "sadar", "ngekos": "tinggal di kos", "ngelamar": "melamar", "ngeliat": "melihat", "ngemeng": "bicara terus-terusan", "ngerti": "mengerti", "nggak": "tidak", "ngikut": "ikut", "nginep": "menginap", "ngisi": "mengisi", "ngmg": "bicara", "ngocol": "lucu", "ngomongin": "membicarakan", "ngumpul": "berkumpul", "ni": "ini", "nyasar": "tersesat", "nyariin": "mencari", "nyiapin": "mempersiapkan", "nyiram": "menyiram", "nyok": "ayo", "o/": "oleh", "ok": "ok", "priksa": "periksa", "pro": "profesional", "psn": "pesan", "psti": "pasti", "puanas": "panas", "qmo": "kamu", "qt": "kita", "rame": "ramai", "raskin": "rakyat miskin", "red": "redaksi", "reg": "register", "rejeki": "rezeki", "renstra": "rencana strategis", "reskrim": "reserse kriminal", "sni": "sini", "somse": "sombong sekali", "sorry": "maaf", "sosbud": "sosial-budaya", "sospol": "sosial-politik", "sowry": "maaf", "spd": "sepeda", "sprti": "seperti", "spy": "supaya", "stelah": "setelah", "subbag": "subbagian", "sumbangin": "sumbangkan", "sy": "saya", "syp": "siapa", "tabanas": "tabungan pembangunan nasional", "tar": "nanti", "taun": "tahun", "tawh": "tahu", "tdi": "tadi", "te2p": "tetap", "tekor": "rugi", "telkom": "telekomunikasi", "telp": "telepon", "temen2": "teman-teman", "tengok": "menjenguk", "terbitin": "terbitkan", "tgl": "tanggal", "thanks": "terima kasih", "thd": "terhadap", "thx": "terima kasih", "tipi": "TV", "tkg": "tukang", "tll": "terlalu", "tlpn": "telepon", "tman": "teman", "tmbh": "tambah", "tmn2": "teman-teman", "tmph": "tumpah", "tnda": "tanda", "tnh": "tanah", "togel": "toto gelap", "tp": "tapi", "tq": "terima kasih", "trgntg": "tergantung", "trims": "terima kasih", "cb": "coba", "y": "ya", "munfik": "munafik", "reklamuk": "reklamasi", "sma": "sama", "tren": "trend", "ngehe": "kesal", "mz": "mas", "analisise": "analisis", "sadaar": "sadar", "sept": "september", "nmenarik": "menarik", "zonk": "bodoh", "rights": "benar", "simiskin": "miskin", "ngumpet": "sembunyi", "hardcore": "keras", "akhirx": "akhirnya", "solve": "solusi", "watuk": "batuk", "ngebully": "intimidasi", "masy": "masyarakat", "still": "masih", "tauk": "tahu", "mbual": "bual", "tioghoa": "tionghoa", "ngentotin": "senggama", "kentot": "senggama", "faktakta": "fakta", "sohib": "teman", "rubahnn": "rubah", "trlalu": "terlalu", "nyela": "cela", "heters": "pembenci", "nyembah": "sembah", "most": "paling", "ikon": "lambang", "light": "terang", "pndukung": "pendukung", "setting": "atur", "seting": "akting", "next": "lanjut", "waspadalah": "waspada", "gantengsaya": "ganteng", "parte": "partai", "nyerang": "serang", "nipu": "tipu", "ktipu": "tipu", "jentelmen": "berani", "buangbuang": "buang", "tsangka": "tersangka", "kurng": "kurang", "ista": "nista", "less": "kurang", "koar": "teriak", "paranoid": "takut", "problem": "masalah", "tahi": "kotoran", "tirani": "tiran", "tilep": "tilap", "happy": "bahagia", "tak": "tidak", "penertiban": "tertib", "uasai": "kuasa", "mnolak": "tolak", "trending": "trend", "taik": "tahi", "wkwkkw": "tertawa", "ahokncc": "ahok", "istaa": "nista", "benarjujur": "jujur", "mgkin": "mungkin", "app": "aplikasi", "trimakasih": "terima kasih", "udh": "sudah", "terimakasih": "terima kasih", "apotik": "apotek"}

In [ ]:
# function to convert slangwords into their standard forms
def convert_slangwords(tokens, slangwords):
  fixed_words = [slangwords[word.lower()] if word in slangwords else word for word in tokens]
  return fixed_words

In [ ]:
from nltk.corpus import stopwords
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

# function to remove stopwords in both english and indonesian
def remove_stopwords(tokens):
  # stopwords collected from nltk
  stopwords_idn = list(set(stopwords.words('indonesian')))
  stopwords_eng = list(set(stopwords.words('english')))

  # stopwords collected from sastrawi
  factory = StopWordRemoverFactory()
  stopwords_sastrawi = factory.get_stop_words()

  # additional commonly used stopwords
  additional_stopwords = ['iya','yaa','gak','nya','na','sih','ku',"di","ga","ya","gaa","loh","kah","woi","woii","woy"]

  # combining all extracted stopwords
  merged_stopwords = set(stopwords_idn + stopwords_eng + additional_stopwords + stopwords_sastrawi) # making sure there's no duplicated stopwords

  filtered_text = [text for text in tokens if text not in merged_stopwords]
  return filtered_text

In [ ]:
# function to finalize text preprocessing by calling all function and combine it into a sentence
def final_preprocessing(review, slangwords):
  # text cleaning
  cleaned_text = text_cleaning(review)

  # case folding
  lowercase_text = text_lowercase(cleaned_text)

  # tokenization
  tokens = tokenization(lowercase_text)

  # slangwords fixing
  fixed_words = convert_slangwords(tokens, slangwords)

  # stopwords removal
  filtered_tokens = remove_stopwords(fixed_words)

  # combining stems to a final string
  final_result = ' '.join(filtered_tokens)

  return cleaned_text, lowercase_text, tokens, fixed_words, filtered_tokens, final_result

In [ ]:
# function call
review_data['preprocessedReview'] = review_data['content'].apply(lambda review:
                                                                 final_preprocessing(review, slangwords)[5]) # only adding the final result to the df

In [ ]:
# final df
review_data

,userName,content,score,at,preprocessedReview
0,Pengguna Google,Dokternya baik,5,2025-01-21 01:11:20,dokternya
1,Pengguna Google,Dokter yg ramah ramah dan obat yg di anjurkan ...,5,2025-01-21 00:38:40,dokter ramah ramah obat anjurkan cocok
2,Pengguna Google,Deskripsi dokter kurang detail .. masak sudah ...,2,2025-01-18 11:50:03,deskripsi dokter detail masak bayar salah dokt...
3,Pengguna Google,Konsul dengan dokter M.Ridwan hakiki enak bgt ...,5,2025-01-16 02:22:16,konsul dokter mridwan hakiki enak banget diagn...
4,Pengguna Google,Responsif dan bagus pelayanannya,5,2025-01-15 22:11:22,responsif bagus pelayanannya
...,...,...,...,...,...
10995,Pengguna Google,Aplikasi yg bagus karena langsung di jwb sama ...,5,2018-09-13 09:04:01,aplikasi bagus langsung dokter
10996,Pengguna Google,Terimaksih sudah membuat aplikasi yg berguna d...,5,2018-09-13 07:08:03,terimaksih aplikasi berguna bermanfaatsukses h...
10997,Pengguna Google,Sangat membantu mendekatkan masyarakat dengan ...,5,2018-09-13 02:45:53,membantu mendekatkan masyarakat layanan keseha...
10998,Pengguna Google,"Terimakasih, Sangat membantu, semoga kedepanny...",5,2018-09-12 20:42:15,terima kasih membantu semoga kedepannya maju


## **Data Labeling**

In [ ]:
import csv
import requests
from io import StringIO

# function to access lexicon data from url
def lexicon_data(url):
  lexicon_dict = dict()

  response = requests.get(url)

  if response.status_code == 200:
    url_reader = csv.reader(StringIO(response.text), delimiter=',')

    for data in url_reader:
      lexicon_dict[data[0]] = int(data[1])
  else:
    print('Unsuccessfully fetched data')

  return lexicon_dict

In [ ]:
# import positive and negative lexicons from gitHub
positive_lexicon = lexicon_data('https://raw.githubusercontent.com/angelmetanosaa/dataset/main/lexicon_positive.csv')
negative_lexicon = lexicon_data('https://raw.githubusercontent.com/angelmetanosaa/dataset/main/lexicon_negative.csv')

In [ ]:
# function to determine sentiment polarity
def det_sentiment_polarity(text, positive_lexicon, negative_lexicon):
  sentiment_score = 0

  # calculate sentiment score based on lexicons
  for word in text.split():
    if word in positive_lexicon:
      sentiment_score += positive_lexicon[word]
    elif word in negative_lexicon:
      sentiment_score += negative_lexicon[word]
    else:
      pass

  # check final score
  if sentiment_score > 0:
    sentiment_polarity = 'positive'
  elif sentiment_score < 0:
    sentiment_polarity = 'negative'
  else:
    sentiment_polarity = 'neutral'

  return sentiment_score, sentiment_polarity

In [ ]:
# create new columns to store the sentiment score and the polarity of each data
review_data[['sentimentScore', 'sentimentPolarity']] = review_data['preprocessedReview'].apply(
    lambda review: pd.Series(det_sentiment_polarity(review, positive_lexicon, negative_lexicon))
    )

review_data

,userName,content,score,at,preprocessedReview,sentimentScore,sentimentPolarity
0,Pengguna Google,Dokternya baik,5,2025-01-21 01:11:20,dokternya,0,neutral
1,Pengguna Google,Dokter yg ramah ramah dan obat yg di anjurkan ...,5,2025-01-21 00:38:40,dokter ramah ramah obat anjurkan cocok,11,positive
2,Pengguna Google,Deskripsi dokter kurang detail .. masak sudah ...,2,2025-01-18 11:50:03,deskripsi dokter detail masak bayar salah dokt...,-9,negative
3,Pengguna Google,Konsul dengan dokter M.Ridwan hakiki enak bgt ...,5,2025-01-16 02:22:16,konsul dokter mridwan hakiki enak banget diagn...,14,positive
4,Pengguna Google,Responsif dan bagus pelayanannya,5,2025-01-15 22:11:22,responsif bagus pelayanannya,2,positive
...,...,...,...,...,...,...,...
10995,Pengguna Google,Aplikasi yg bagus karena langsung di jwb sama ...,5,2018-09-13 09:04:01,aplikasi bagus langsung dokter,1,positive
10996,Pengguna Google,Terimaksih sudah membuat aplikasi yg berguna d...,5,2018-09-13 07:08:03,terimaksih aplikasi berguna bermanfaatsukses h...,5,positive
10997,Pengguna Google,Sangat membantu mendekatkan masyarakat dengan ...,5,2018-09-13 02:45:53,membantu mendekatkan masyarakat layanan keseha...,12,positive
10998,Pengguna Google,"Terimakasih, Sangat membantu, semoga kedepanny...",5,2018-09-12 20:42:15,terima kasih membantu semoga kedepannya maju,8,positive


In [ ]:
# polarity proportion distribution
review_data['sentimentPolarity'].value_counts(normalize=True)

,proportion
sentimentPolarity,
positive,0.820182
negative,0.133273
neutral,0.046545


## **Data Splitting and Feature Extractions**

In [ ]:
# separate target variable from features
X = review_data['preprocessedReview']
y = review_data['sentimentPolarity']

In [ ]:
# feature extraction using tf-idf
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vec = TfidfVectorizer(
    max_features=300, # selecting only 300 features
    min_df=30,
)
X_tfidf = tfidf_vec.fit_transform(X)

feats_df = pd.DataFrame(X_tfidf.toarray(), columns=tfidf_vec.get_feature_names_out())
feats_df

,aamiin,ahli,akurat,alamat,alhamdulilah,alhamdulillah,allah,aman,anak,antri,antrian,apk,apl,aplikasi,aplikasinya,apotek,asuransi,bagus,baik,balas,banget,bantu,bantuan,barang,bayar,beli,berbayar,berguna,berkat,berkonsultasi,bermanfaat,berobat,berpengalaman,best,biar,biaya,bikin,bilang,bingung,bintang,buka,butuh,cancel,cari,cepat,chat,coba,cocok,covid,cpt,...,sembuh,semoga,senang,seneng,sesi,sesuai,si,sibuk,simple,singkat,situasi,skrg,sm,sngat,solusi,solusinya,sopan,spesialis,suka,sukses,sulit,sungguh,super,suruh,susah,sya,takut,tanggal,tanggap,tanggapan,telepon,terbaik,terbantu,terdekat,terima,terimaksih,terjangkau,terpercaya,tersedia,thank,tinggal,tingkatkan,top,trus,tunggu,tutup,uang,urgent,vaksin,via
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00000,0.0,0.0,0.000000,0.0,0.0,0.0,0.00000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00000,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00000,0.0,0.0,0.000000,0.0,0.0,0.0,0.00000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.61769,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00000,0.0,0.0,0.000000,0.0,0.0,0.0,0.44804,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.282142,0.0,0.0,0.0,0.00000,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.31311,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00000,0.0,0.0,0.266278,0.0,0.0,0.0,0.00000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00000,0.0,0.0,...,0.453555,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.482089,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.39822,0.0,0.0,0.000000,0.0,0.0,0.0,0.00000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00000,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.431609,0.0,0.0,0.0,0.53118,0.0,0.0,0.000000,0.0,0.0,0.0,0.00000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00000,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0
10996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

In [ ]:
# train test split
from sklearn.model_selection import train_test_split

def split_data(X, y, test_size, random_state, shuffle=True):
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state, shuffle=shuffle)
  return X_train, X_test, y_train, y_test

## **Modeling**

In [ ]:
from sklearn.metrics import accuracy_score

# function to train models
def model_training(model, X_train, X_test, y_train, y_test):
  model = model
  model.fit(X_train.toarray(), y_train)

  # model prediction
  y_pred_train = model.predict(X_train.toarray())
  y_pred_test = model.predict(X_test.toarray())

  # model evaluation
  accuracy_train = accuracy_score(y_pred_train, y_train)
  accuracy_test = accuracy_score(y_pred_test, y_test)

  result = {
      'accuracy_train': round(accuracy_train, 4),
      'accuracy_test': round(accuracy_test, 4)
  }

  return result

In [ ]:
# 2 schemes: data split 70/30 and 80/20
X_train_30, X_test_30, y_train_30, y_test_30 = split_data(X_tfidf, y, test_size=0.3, random_state=42, shuffle=False) # for 70/30 split
X_train_20, X_test_20, y_train_20, y_test_20 = split_data(X_tfidf, y, test_size=0.2, random_state=42, shuffle=False) # for 80/20 split

### **Logistic Regression**

In [ ]:
from sklearn.linear_model import LogisticRegression

# model initialization
log_reg = LogisticRegression()

# function call
result_lr_30 = model_training(log_reg, X_train_30, X_test_30, y_train_30, y_test_30)
result_lr_20 = model_training(log_reg, X_train_20, X_test_20, y_train_20, y_test_20)

# saving results for summarization purpose
lr_results = {
    'Model': 'Logistic Regression',
    'Accuracy Train (70:30)': result_lr_30['accuracy_train'],
    'Accuracy Test (70:30)': result_lr_30['accuracy_test'],
    'Accuracy Train (80:20)': result_lr_20['accuracy_train'],
    'Accuracy Test (80:20)': result_lr_20['accuracy_test']
}

# print training result
print(f'Logistic Regression Training Result:\n')
print(f"Train data accuracy score for data split 70/30: {result_lr_30['accuracy_train']}")
print(f"Test data accuracy score for data split 70/30: {result_lr_30['accuracy_test']}")
print('vs.')
print(f"Train data accuracy score for data split 80/20: {result_lr_20['accuracy_train']}")
print(f"Test data accuracy score for data split 80/20: {result_lr_20['accuracy_test']}")

Logistic Regression Training Result:

Train data accuracy score for data split 70/30: 0.909
Test data accuracy score for data split 70/30: 0.9055
vs.
Train data accuracy score for data split 80/20: 0.9103
Test data accuracy score for data split 80/20: 0.8973


### **Random Forest Classifier**

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# model initialization
rf_cls = RandomForestClassifier(n_estimators=100)

# function call
result_rf_30 = model_training(rf_cls, X_train_30, X_test_30, y_train_30, y_test_30)
result_rf_20 = model_training(rf_cls, X_train_20, X_test_20, y_train_20, y_test_20)

# saving results for summarization purpose
rf_results = {
    'Model': 'Random Forest Classifier',
    'Accuracy Train (70:30)': result_rf_30['accuracy_train'],
    'Accuracy Test (70:30)': result_rf_30['accuracy_test'],
    'Accuracy Train (80:20)': result_rf_20['accuracy_train'],
    'Accuracy Test (80:20)': result_rf_20['accuracy_test']
}

# print training result
print(f'Random Forest Classifier Training Result:\n')
print(f"Train data accuracy score for data split 70/30: {result_rf_30['accuracy_train']}")
print(f"Test data accuracy score for data split 70/30: {result_rf_30['accuracy_test']}")
print('vs.')
print(f"Train data accuracy score for data split 80/20: {result_rf_20['accuracy_train']}")
print(f"Test data accuracy score for data split 80/20: {result_rf_20['accuracy_test']}")

Random Forest Classifier Training Result:

Train data accuracy score for data split 70/30: 0.9875
Test data accuracy score for data split 70/30: 0.8812
vs.
Train data accuracy score for data split 80/20: 0.9874
Test data accuracy score for data split 80/20: 0.8745


### **Naive Bayes**

In [ ]:
from sklearn.naive_bayes import MultinomialNB

# model initialization
naive_bayes = MultinomialNB()

# function call
result_nb_30 = model_training(naive_bayes, X_train_30, X_test_30, y_train_30, y_test_30)
result_nb_20 = model_training(naive_bayes, X_train_20, X_test_20, y_train_20, y_test_20)

# saving results for summarization purpose
nb_results = {
    'Model': 'Naive Bayes',
    'Accuracy Train (70:30)': result_nb_30['accuracy_train'],
    'Accuracy Test (70:30)': result_nb_30['accuracy_test'],
    'Accuracy Train (80:20)': result_nb_20['accuracy_train'],
    'Accuracy Test (80:20)': result_nb_20['accuracy_test']
}

# print training result
print(f'Naive Bayes Training Result:\n')
print(f"Train data accuracy score for data split 70/30: {result_nb_30['accuracy_train']}")
print(f"Test data accuracy score for data split 70/30: {result_nb_30['accuracy_test']}")
print('vs.')
print(f"Train data accuracy score for data split 80/20: {result_nb_20['accuracy_train']}")
print(f"Test data accuracy score for data split 80/20: {result_nb_20['accuracy_test']}")

Naive Bayes Training Result:

Train data accuracy score for data split 70/30: 0.8447
Test data accuracy score for data split 70/30: 0.8464
vs.
Train data accuracy score for data split 80/20: 0.8488
Test data accuracy score for data split 80/20: 0.8341


### **SVM**

In [ ]:
from sklearn.svm import SVC

# model initialization
svm = SVC()

# function call
result_svm_30 = model_training(svm, X_train_30, X_test_30, y_train_30, y_test_30)
result_svm_20 = model_training(svm, X_train_20, X_test_20, y_train_20, y_test_20)

# saving results for summarization purpose
svm_results = {
    'Model': 'SVM',
    'Accuracy Train (70:30)': result_svm_30['accuracy_train'],
    'Accuracy Test (70:30)': result_svm_30['accuracy_test'],
    'Accuracy Train (80:20)': result_svm_20['accuracy_train'],
    'Accuracy Test (80:20)': result_svm_20['accuracy_test']
}

# print training result
print(f'SVM Training Result:\n')
print(f"Train data accuracy score for data split 70/30: {result_svm_30['accuracy_train']}")
print(f"Test data accuracy score for data split 70/30: {result_svm_30['accuracy_test']}")
print('vs.')
print(f"Train data accuracy score for data split 80/20: {result_svm_20['accuracy_train']}")
print(f"Test data accuracy score for data split 80/20: {result_svm_20['accuracy_test']}")

SVM Training Result:

Train data accuracy score for data split 70/30: 0.9383
Test data accuracy score for data split 70/30: 0.9079
vs.
Train data accuracy score for data split 80/20: 0.9385
Test data accuracy score for data split 80/20: 0.9


### **RNN**

In [ ]:
# checking the maximum and the minimum length of review
max_review_length = max(len(data.split()) for data in review_data['preprocessedReview'])
min_review_length = min(len(data.split()) for data in review_data['preprocessedReview'])
print(f'The maximum length of word in review: {max_review_length}')
print(f'The minimum length of word in review: {min_review_length}')

The maximum length of word in review: 62
The minimum length of word in review: 0


In [ ]:
# viewing the empty review (after preprocessed)
review_data.loc[review_data['preprocessedReview'].str.strip() == '']

,userName,content,score,at,preprocessedReview,sentimentScore,sentimentPolarity
337,Pengguna Google,Tepat waktu antar nya,5,2024-08-26 07:21:02,,0,neutral
1124,Pengguna Google,𝑟𝑒𝑠𝑝𝑜𝑛 𝑑𝑟 𝑑𝑜𝑘 𝑠𝑝𝑒𝑐𝑖𝑎𝑙𝑖𝑠 𝑠𝑎𝑛𝑔𝑎𝑡 𝑟𝑎𝑚𝑎ℎ 𝑑𝑎𝑛 𝑐𝑒𝑝𝑎𝑡,5,2023-09-07 13:30:55,,0,neutral


In [ ]:
# remove empty review data
review_data = review_data.loc[review_data['preprocessedReview'].str.strip() != '']

min_review_length = min(len(data.split()) for data in review_data['preprocessedReview'])
print(f'The minimum length of word in review: {min_review_length}')

The minimum length of word in review: 1


In [ ]:
# reupdating X and y
X = review_data['preprocessedReview']
y = review_data['sentimentPolarity']

In [ ]:
# label encoding
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
y = encoder.fit_transform(y)
y

array([1, 2, 0, ..., 2, 2, 2])

In [ ]:
# data preparation
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence

# tokenizing review
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(X)
int_sequences = tokenizer.texts_to_sequences(X)

# pad sequences
max_length = 200
pad_seq = sequence.pad_sequences(int_sequences, maxlen=max_length, padding='post', truncating='post')

# resplitting training and testing data
X_train, X_test, y_train, y_test = train_test_split(pad_seq, y, test_size=0.3, random_state=42, shuffle=False)
print(f'Training data shape: {X_train.shape}')
print(f'Testing data shape: {X_test.shape}')

Training data shape: (7698, 200)
Testing data shape: (3300, 200)


In [ ]:
# create an RNN model (LSTM)
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True, verbose=1) # stop training when val_loss didn't improve

# LSTM model
lstm_model = Sequential([
    Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=300, input_length=max_length),
    LSTM(100),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax'),
])

lstm_model.compile(optimizer='adam',
                   loss='sparse_categorical_crossentropy',
                   metrics=['accuracy'])

# model training
lstm_model.fit(X_train, y_train, epochs=10, batch_size=50, callbacks=[early_stopping], validation_data=(X_test, y_test), verbose=2)

Epoch 1/10
154/154 - 77s - 502ms/step - accuracy: 0.8133 - loss: 0.6047 - val_accuracy: 0.8276 - val_loss: 0.5611
Epoch 2/10
154/154 - 81s - 528ms/step - accuracy: 0.8172 - loss: 0.5885 - val_accuracy: 0.8276 - val_loss: 0.5686
Epoch 3/10
154/154 - 81s - 529ms/step - accuracy: 0.8172 - loss: 0.5878 - val_accuracy: 0.8276 - val_loss: 0.5613
Epoch 4/10
154/154 - 69s - 449ms/step - accuracy: 0.8172 - loss: 0.5872 - val_accuracy: 0.8276 - val_loss: 0.5706
Epoch 4: early stopping
Restoring model weights from the end of the best epoch: 1.


In [ ]:
# model evaluation
lstm_model.evaluate(X_train, y_train, batch_size=1)

7698/7698 ━━━━━━━━━━━━━━━━━━━━ 262s 34ms/step - accuracy: 0.8006 - loss: 0.6224


[0.5797476172447205, 0.8172252774238586]

## **Model Training Summary**

In [ ]:
# dictionary containing model training results
model_results = {
    'Logistic Regression': lr_results,
    'Random Forest Classifier': rf_results,
    'Naive Bayes': nb_results,
    'SVM': svm_results,
}

# create a dataframe to summarize every model's training result
summary_df =pd.DataFrame.from_dict(model_results, orient='index').reset_index(drop=True)
summary_df.rename(columns={'Model': 'Model'}, inplace=True)
summary_df

,Model,Accuracy Train (70:30),Accuracy Test (70:30),Accuracy Train (80:20),Accuracy Test (80:20)
0,Logistic Regression,0.9090,0.9055,0.9103,0.8973
1,Random Forest Classifier,0.9875,0.8812,0.9874,0.8745
2,Naive Bayes,0.8447,0.8464,0.8488,0.8341
3,SVM,0.9383,0.9079,0.9385,0.9000
